In [1]:
from scipy.io import wavfile
from scipy.fft import fft, fftfreq
import numpy as np
import matplotlib.pyplot as plt
import math
import datetime
import os
import threading

In [2]:
def save_fft_data(song_name, frame_length=10, n_threads=8):

    filepath = f"./songs/{song_name}.wav"

    # load wav
    print(f"Loading {filepath}")
    samplerate, data = wavfile.read(filepath)
    bin_step = math.floor(samplerate / 100)

    # make dir
    if (not os.path.isdir("./data")):
        print("Creating directory ./data/")
        os.mkdir("./data/")
    if (not os.path.isdir("./data/fft")):
        print("Creating directory ./data/fft/")
        os.mkdir("./data/fft/")
    if (not os.path.isdir(f"./data/fft/{song_name}/")):
        print(f"Creating directory ./data/fft/{song_name}/")
        os.mkdir(f"./data/fft/{song_name}/")
    
    print(f"Saving data to ./{song_name}/")

    # save one frame of data
    def save_frame(start_bin, thread_i):
        fft_bins = []
        end_bin = start_bin+math.floor(samplerate/bin_step*frame_length)
        for i_bin in range(start_bin, end_bin):
            sample_start = i_bin*bin_step
            sample_end = sample_start + samplerate
            bin = data[sample_start:sample_end,:]

            # take ffts
            channel_0_fft = np.abs(fft(bin[:,0])[1:samplerate//2])
            channel_1_fft = np.abs(fft(bin[:,1])[1:samplerate//2])

            fft_bins.append(np.mean(np.array([channel_0_fft, channel_1_fft]), axis=0))

        print(f"| thread_{thread_i}:\ttime processed: {datetime.timedelta(seconds=(end_bin*bin_step)/samplerate)}")
        np.save(f"./data/fft/{song_name}/{start_bin*bin_step}-{end_bin*bin_step}.npy", np.clip(np.log(np.array(fft_bins))/20, 0, 1))

    # multithreading function
    def thread_task_frame(bins, thread_i):
        for bin in bins:
            save_frame(bin, thread_i)
        print(f"thread_{thread_i} finished.")

    threads = []
    n_bins = math.ceil((data.shape[0]-samplerate)/bin_step)
    thread_bins = [[] for _ in range(n_threads)]
    for i in range(n_bins//math.floor(samplerate/bin_step*frame_length)):
        thread_bins[i%n_threads] += [i*math.floor(samplerate/bin_step*frame_length)]

    for bins in thread_bins:
        thread_index = len(threads)
        print(f"Tasking thread_{thread_index} to {bins}")
        threads.append(threading.Thread(target=thread_task_frame, args=(bins, thread_index)))
        threads[thread_index].start()

    for i in range(n_threads):
        threads[i].join()
        print(f"Joining thread_{i}")

    return 0
    

In [3]:
def sort_key(element):
    try:
        return int(element.split("-")[0])
    except:
        return -1

def combine_files(directory):
    full_frames = []
    files = os.listdir(f"./data/fft/{directory}/")
    files.sort(key=sort_key)
    for filename in files:
        filepath = f"./data/fft/{directory}/{filename}"
        if not os.path.isfile(filepath) or not filename[-3:]=='npy': continue
        data = np.load(filepath)
        full_frames += [data]
    np.save(f"{directory}.npy", np.array(full_frames).reshape(-1, 23999))

In [4]:
save_fft_data("test_song", frame_length=10, n_threads=8)

Loading ./songs/test_song.wav
Creating directory ./data/
Creating directory ./data/fft/
Creating directory ./data/fft/test_song/
Saving data to ./test_song/
Tasking thread_0 to [0, 8000, 16000]
Tasking thread_1 to [1000, 9000, 17000]
Tasking thread_2 to [2000, 10000, 18000]
Tasking thread_3 to [3000, 11000, 19000]
Tasking thread_4 to [4000, 12000, 20000]
Tasking thread_5 to [5000, 13000]
Tasking thread_6 to [6000, 14000]
Tasking thread_7 to [7000, 15000]
| thread_5:	time processed: 0:01:00
| thread_4:	time processed: 0:00:50
| thread_1:	time processed: 0:00:20
| thread_3:	time processed: 0:00:40
| thread_2:	time processed: 0:00:30
| thread_6:	time processed: 0:01:10
| thread_0:	time processed: 0:00:10
| thread_7:	time processed: 0:01:20
| thread_5:	time processed: 0:02:20
| thread_4:	time processed: 0:02:10
| thread_3:	time processed: 0:02:00
| thread_1:	time processed: 0:01:40
| thread_0:	time processed: 0:01:30
| thread_7:	time processed: 0:02:40
| thread_6:	time processed: 0:02:30
|

0

In [5]:
combine_files("test_song")
plt.imsave("test_song.png", np.load("test_song.npy").transpose())

In [6]:
save_fft_data("classical", frame_length=10, n_threads=8)

Loading ./songs/classical.wav
Creating directory ./data/fft/classical/
Saving data to ./classical/
Tasking thread_0 to [0, 8000, 16000, 24000, 32000, 40000, 48000, 56000, 64000, 72000, 80000, 88000, 96000, 104000, 112000, 120000, 128000, 136000, 144000, 152000, 160000, 168000, 176000, 184000, 192000, 200000, 208000, 216000, 224000, 232000, 240000, 248000, 256000, 264000, 272000, 280000, 288000, 296000, 304000, 312000, 320000, 328000, 336000, 344000, 352000, 360000, 368000, 376000, 384000, 392000, 400000, 408000, 416000, 424000, 432000, 440000, 448000, 456000, 464000, 472000, 480000, 488000, 496000, 504000, 512000, 520000, 528000, 536000, 544000, 552000, 560000, 568000, 576000, 584000, 592000, 600000, 608000, 616000, 624000, 632000, 640000, 648000, 656000, 664000, 672000, 680000, 688000, 696000, 704000, 712000, 720000, 728000, 736000, 744000, 752000, 760000, 768000, 776000, 784000, 792000, 800000, 808000, 816000, 824000, 832000, 840000, 848000, 856000, 864000, 872000, 880000, 888000, 89

0